# Programmatically Replicate Gating from a FlowJo Workspace

In [ ]:
import os
import bokeh
from bokeh.plotting import show
import matplotlib.pyplot as plt
import numpy as np

import flowkit as fk

bokeh.io.output_notebook()
%matplotlib inline

## Load FlowJo Workspace

We load the workspace just to show the gating strategy that we want to replicate programmatically. 

**At the end of the notebook, the replicated analysis is exported to a new FlowJo 10 workspace.**

In [ ]:
base_dir = "../../../data/8_color_data_set"

sample_path = os.path.join(base_dir, "fcs_files")
wsp_path = os.path.join(base_dir, "8_color_ICS.wsp")

In [ ]:
fks_fj = fk.Session(sample_path)
fks_fj.import_flowjo_workspace(wsp_path)

In [ ]:
fks_fj.get_sample_ids()

In [ ]:
fks_fj.get_sample_groups()

### Record our sample ID and sample group for later

In [ ]:
sample_id = '101_DEN084Y5_15_E03_009_clean.fcs'
sample_grp = 'DEN'

In [ ]:
sample = fks_fj.get_sample(sample_id)

In [ ]:
sample.channels

### Here's the gate hierachy we aim to replicate

In [ ]:
print(fks_fj.get_gate_hierarchy(sample_grp, 'ascii'))

In [ ]:
fks_fj.get_group_transforms(sample_grp)

In [ ]:
fks_fj.analyze_samples(sample_grp)

In [ ]:
results = fks_fj.get_gating_results(sample_grp, sample_id)

In [ ]:
fks_fj.get_gate_membership(sample_grp, sample_id, 'Singlets')

In [ ]:
results.report.head()

## Create a new Session

Then, load a single FCS sample and define a new sample group.

In [ ]:
fks = fk.Session()

full_sample_path = os.path.join(sample_path, sample_id)

sample_grp = 'DEN'

fks.add_sample_group(sample_grp)
fks.add_samples(full_sample_path, group_name=sample_grp)

Load the compensation matrix from a CSV file and add it to the group

In [ ]:
# setup comp matrix
detectors = [sample.pnn_labels[i] for i in sample.fluoro_indices]
den_comp_mat = fk.Matrix('den_comp', '../../../examples/data/8_color_data_set/den_comp.csv', detectors)

In [ ]:
den_comp_mat.detectors

In [ ]:
fks.add_comp_matrix(den_comp_mat, group_name=sample_grp)

Define our transformations and add them to the group

In [ ]:
# Setup transforms
time_xform = fk.transforms.LinearTransform('time_xform', param_t=72.0, param_a=0.8511997311)
scatter_xform = fk.transforms.LinearTransform('lin_xform', param_t=262144, param_a=0)
flr_xform = fk.transforms.LogicleTransform(
    'logicle_xform', 
    param_t=262144.0, 
    param_w=1.0, 
    param_m=4.418539922,
    param_a=0.0
)
fks.add_transform(time_xform, group_name=sample_grp)
fks.add_transform(scatter_xform, group_name=sample_grp)
fks.add_transform(flr_xform, group_name=sample_grp)

### Begin defining gates, starting with the Time gate

In [ ]:
# NOTE: The ranges we use for this time gate are not the same as those in the .wsp file, 
#       so event counts will be different
time_dim = fk.Dimension(
    'Time', 
    compensation_ref='uncompensated', 
    transformation_ref=time_xform.id, 
    range_min=0.1, 
    range_max=0.96
)

In [ ]:
# create time gate using Time dimension
time_gate = fk.gates.RectangleGate('Time', None, [time_dim])

In [ ]:
fks.add_gate(time_gate, group_name=sample_grp)

In [ ]:
print(fks.get_gate_hierarchy(sample_grp))

In [ ]:
fks.analyze_samples(sample_grp)

In [ ]:
p = fks.plot_gate(sample_grp, sample_id, "Time")
show(p)

### Define Singlet gate

In [ ]:
# create singlet gate, starting with dimensions
dim_fsc_w = fk.Dimension('FSC-W', 'uncompensated', transformation_ref=scatter_xform.id)
dim_fsc_h = fk.Dimension('FSC-H', 'uncompensated', transformation_ref=scatter_xform.id)

In [ ]:
# look at dimensions gated on Time
p = fks.plot_scatter(
    sample_id, 
    dim_fsc_w, 
    dim_fsc_h, 
    group_name=sample_grp, 
    gate_name="Time", 
    subsample=True
)
show(p)

In [ ]:
# get our vertices from analyst, algo, etc.
singlet_vertices = [
    fk.Vertex([0.328125, 0.1640625]),
    fk.Vertex([0.296875, 0.1484375]),
    fk.Vertex([0.27734375, 0.25390625]),
    fk.Vertex([0.27734375, 0.52734375]),
    fk.Vertex([0.28125, 0.78515625]),
    fk.Vertex([0.30859375, 0.8515625]),
    fk.Vertex([0.34765625, 0.3984375]),
    fk.Vertex([0.3359375, 0.1875])
]

In [ ]:
singlet_gate = fk.gates.PolygonGate(
    'Singlets', 
    'Time', 
    [dim_fsc_w, dim_fsc_h], 
    singlet_vertices
)

In [ ]:
fks.add_gate(singlet_gate, group_name=sample_grp)

In [ ]:
print(fks.get_gate_hierarchy(sample_grp))

In [ ]:
fks.analyze_samples(sample_grp)

### Define Live gate

In [ ]:
# create live cell gate, creating dims then plotting the previous gate
dim_amine_a = fk.Dimension('Aqua Amine FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)
dim_ssc_a = fk.Dimension('SSC-A', 'uncompensated', transformation_ref=scatter_xform.id)

In [ ]:
# look at dimensions gated on Singlets
p = fks.plot_scatter(
    sample_id, 
    dim_amine_a, 
    dim_ssc_a, 
    group_name=sample_grp, 
    gate_name="Singlets", 
    subsample=True,
    x_min=0,
    x_max=1.0,
    y_min=0,
    y_max=1.0
)
show(p)

In [ ]:
# get our vertices from analyst, algo, etc.
live_cell_vertices = [
    fk.Vertex([0.2629268137285685, 0.0625]),
    fk.Vertex([0.24318837264468562, 0.03515625]),
    fk.Vertex([0.21573453285608676, 0.0390625]),
    fk.Vertex([0.20396768438347745, 0.0546875]),
    fk.Vertex([0.20396768438347745, 0.140625]),
    fk.Vertex([0.20460078058895426, 0.3117570495605469]),
    fk.Vertex([0.2355517136894538, 0.328125]),
    fk.Vertex([0.26856506770333155, 0.3125]),
    fk.Vertex([0.29042797365869377, 0.24609375]),
    fk.Vertex([0.29042797365869377, 0.1484375])
]

In [ ]:
live_cell_gate = fk.gates.PolygonGate(
    'aAmine-', 
    'Singlets', 
    [dim_amine_a, dim_ssc_a], 
    live_cell_vertices
)

In [ ]:
fks.add_gate(live_cell_gate, group_name=sample_grp)

In [ ]:
print(fks.get_gate_hierarchy(sample_grp))

In [ ]:
fks.analyze_samples(sample_grp)

### Define CD3+ gate

In [ ]:
# create CD3+ gate
dim_cd3_a = fk.Dimension(
    'CD3 APC-H7 FLR-A', 
    den_comp_mat.id, 
    transformation_ref=flr_xform.id
)

In [ ]:
f = fks.plot_scatter(
    sample_id, 
    dim_cd3_a, 
    dim_ssc_a, 
    group_name=sample_grp, 
    gate_name='aAmine-',
    x_min=0,
    x_max=1,
    y_min=0,
    y_max=1,
    subsample=True
)
show(f)

In [ ]:
# get our vertices from analyst, algo, etc.
cd3_vertices = [
    fk.Vertex([0.28415161867527605, 0.11328125]),
    fk.Vertex([0.3132637699981912, 0.203125]),
    fk.Vertex([0.42207818508379846, 0.3046875]),
    fk.Vertex([0.5067109372185516, 0.359375]),
    fk.Vertex([0.6853991917182599, 0.35546875]),
    fk.Vertex([0.6896802981119161, 0.05078125]),
    fk.Vertex([0.5692952580886116, 0.01953125]),
    fk.Vertex([0.3192472844795108, 0.01953125])
]

In [ ]:
cd3_gate = fk.gates.PolygonGate('CD3+', 'aAmine-', [dim_cd3_a, dim_ssc_a], cd3_vertices)

In [ ]:
fks.add_gate(cd3_gate, group_name=sample_grp)

In [ ]:
fks.analyze_samples(sample_grp)

### Define CD4+ and CD8+ branch gates

In [ ]:
# create CD4+ & CD8+ gates
dim_cd4_a = fk.Dimension('CD4 PE-Cy7 FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)
dim_cd8_a = fk.Dimension('CD8 PerCP-Cy55 FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)

In [ ]:
# look at dimensions gated on CD3+
p = fks.plot_scatter(
    sample_id, 
    dim_cd4_a, 
    dim_cd8_a, 
    group_name=sample_grp, 
    gate_name="CD3+", 
    subsample=True,
    x_min=0,
    x_max=1.0,
    y_min=0,
    y_max=1.0
)
show(p)

In [ ]:
cd4_vertices = [
    fk.Vertex([0.33228361583463906, 0.20521609423858533]),
    fk.Vertex([0.32558526100158003, 0.22402959677045098]),
    fk.Vertex([0.3288891623611386, 0.2534446627500065]),
    fk.Vertex([0.35453955634069056, 0.3162117257472119]),
    fk.Vertex([0.38884335063325615, 0.37109236044857546]),
    fk.Vertex([0.49681261945848476, 0.39344265440087484]),
    fk.Vertex([0.6241875538107384, 0.4172035843426509]),
    fk.Vertex([0.6811074648618941, 0.32237152038011546]),
    fk.Vertex([0.6939511366527197, 0.23138424146124928]),
    fk.Vertex([0.6982119969313532, 0.20396768438347745]),
    fk.Vertex([0.5311666680646416, 0.20396768438347745]),
    fk.Vertex([0.33576875548246565, 0.20396768438347745])
]

In [ ]:
cd8_vertices = [
    fk.Vertex([0.19654236830112726, 0.8063681300583732]),
    fk.Vertex([0.7981838566398077, 0.8186148712026381]),
    fk.Vertex([0.8145363952765393, 0.45683210068669505]),
    fk.Vertex([0.36683425557526916, 0.37109236044857546]),
    fk.Vertex([0.28415161867527605, 0.2949602838822682]),
    fk.Vertex([0.19654236830112726, 0.28826829762740075])
]

In [ ]:
cd4_gate = fk.gates.PolygonGate('CD4+', 'CD3+', [dim_cd4_a, dim_cd8_a], cd4_vertices)
cd8_gate = fk.gates.PolygonGate('CD8+', 'CD3+', [dim_cd4_a, dim_cd8_a], cd8_vertices)

In [ ]:
fks.add_gate(cd4_gate, group_name=sample_grp)
fks.add_gate(cd8_gate, group_name=sample_grp)

In [ ]:
print(fks.get_gate_hierarchy(sample_grp))

In [ ]:
fks.analyze_samples(sample_grp)

### Create CD107a+, IFNg, IL2, & TNFa gates, then add to CD4 & CD8 branches

In [ ]:
dim_cd107a_a = fk.Dimension('CD107a PE FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)
dim_ifng_a = fk.Dimension('IFNg APC FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)
dim_il2_a = fk.Dimension('IL2 BV421 FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)
dim_tnfa_a = fk.Dimension('TNFa FITC FLR-A', den_comp_mat.id, transformation_ref=flr_xform.id)

In [ ]:
# start with CD107a gated on CD4+ branch
p = fks.plot_scatter(
    sample_id, 
    dim_cd3_a, 
    dim_cd107a_a, 
    group_name=sample_grp, 
    gate_name="CD4+", 
    subsample=True,
    x_min=0,
    x_max=1.0,
    y_min=0,
    y_max=1.0
)
show(p)

In [ ]:
# these next gates are all rectangle gates, so bounds will be defined in the Dimension instances
cd4_cd3_dim = fk.Dimension(
    'CD3 APC-H7 FLR-A', 
    compensation_ref=den_comp_mat.id, 
    transformation_ref=flr_xform.id, 
    range_min=0.2, 
    range_max=1.0
)
cd4_cd107_dim = fk.Dimension(
    'CD107a PE FLR-A', 
    compensation_ref=den_comp_mat.id, 
    transformation_ref=flr_xform.id, 
    range_min=0.5645990565096747, 
    range_max=1.1647085183977386
)

In [ ]:
cd4_cd107_gate = fk.gates.RectangleGate('CD107a+', 'CD4+', [cd4_cd3_dim, cd4_cd107_dim])

In [ ]:
fks.add_gate(cd4_cd107_gate, group_name=sample_grp)

In [ ]:
# the IFNg, IL2, & TNFa gates will share the same CD3 bounds, so only define their dims
cd4_ifng_dim = fk.Dimension(
    'IFNg APC FLR-A', 
    compensation_ref=den_comp_mat.id, 
    transformation_ref=flr_xform.id, 
    range_min=0.30232120911824945, 
    range_max=1.1874194728667935
)
cd4_il2_dim = fk.Dimension(
    'IL2 BV421 FLR-A', 
    compensation_ref=den_comp_mat.id, 
    transformation_ref=flr_xform.id, 
    range_min=0.45683210351871706, 
    range_max=1.136312449258137
)
cd4_tnfa_dim = fk.Dimension(
    'TNFa FITC FLR-A', 
    compensation_ref=den_comp_mat.id, 
    transformation_ref=flr_xform.id, 
    range_min=0.2904279740369533, 
    range_max=1.0965394616932618
)

In [ ]:
# and likewise, create & add their rectangle gates
cd4_ifng_gate = fk.gates.RectangleGate('IFNg+', 'CD4+', [cd4_cd3_dim, cd4_ifng_dim])
cd4_il2_gate = fk.gates.RectangleGate('IL2+', 'CD4+', [cd4_cd3_dim, cd4_il2_dim])
cd4_tnfa_gate = fk.gates.RectangleGate('TNFa+', 'CD4+', [cd4_cd3_dim, cd4_tnfa_dim])

fks.add_gate(cd4_ifng_gate, group_name=sample_grp)
fks.add_gate(cd4_il2_gate, group_name=sample_grp)
fks.add_gate(cd4_tnfa_gate, group_name=sample_grp)

In [ ]:
# the gates in the CD8+ branch are the same except for the parent ID
cd8_cd107_gate = fk.gates.RectangleGate('CD107a+', 'CD8+', [cd4_cd3_dim, cd4_cd107_dim])
cd8_ifng_gate = fk.gates.RectangleGate('IFNg+', 'CD8+', [cd4_cd3_dim, cd4_ifng_dim])
cd8_il2_gate = fk.gates.RectangleGate('IL2+', 'CD8+', [cd4_cd3_dim, cd4_il2_dim])
cd8_tnfa_gate = fk.gates.RectangleGate('TNFa+', 'CD8+', [cd4_cd3_dim, cd4_tnfa_dim])

In [ ]:
fks.add_gate(cd8_cd107_gate, group_name=sample_grp)
fks.add_gate(cd8_ifng_gate, group_name=sample_grp)
fks.add_gate(cd8_il2_gate, group_name=sample_grp)
fks.add_gate(cd8_tnfa_gate, group_name=sample_grp)

In [ ]:
print(fks.get_gate_hierarchy(sample_grp))

In [ ]:
fks.analyze_samples(sample_grp)

### Review all our gates

In [ ]:
for i, row in results.report.iterrows():    
    p = fks.plot_gate(
        sample_grp, 
        row['sample'], # 'sample' is a Pandas, so lookup explicitly
        gate_name=row.gate_name,
        gate_path=row.gate_path,
        x_min=0, 
        x_max=1.2, 
        y_min=0, 
        y_max=1.2
    )
    show(p)

### Get gating results (currently with only 1 sample assigned & analyzed)

In [ ]:
results = fks.get_gating_results(sample_grp, sample_id)

In [ ]:
results.report.head()

### Now add more samples and simply re-run the analysis

In [ ]:
fks.add_samples([
    os.path.join(sample_path, '101_DEN084Y5_15_E01_008_clean.fcs'),
    os.path.join(sample_path, '101_DEN084Y5_15_E05_010_clean.fcs')
    ],
    sample_grp
)

In [ ]:
# running with verbose=True to see progress as it can take some time to run
fks.analyze_samples(sample_grp, verbose=True)

### Retrieve and review the final report

In [ ]:
grp_report = fks.get_group_report(sample_grp)

In [ ]:
grp_report.head()

In [ ]:
grp_report[grp_report.gate_name == 'CD8+']

### Export our sample group analysis to a new FlowJo workspace

In [ ]:
new_wsp_path = os.path.join("", "new_8_color_ICS.wsp")

fks.export_wsp(new_wsp_path, sample_grp)